# 2. Clip Audio
Extract Audio and Video from sample

## First let's create a sample 
instead of working on whole file

In [51]:
from deepdub_sentence import DeepdubSentence
from moviepy.editor import *
from IPython import display

import os
import numpy as np
import pandas as pd
from pathlib import Path
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_audio
from moviepy.video.io.ffmpeg_tools import ffmpeg_merge_video_audio

In [64]:
BASE_DIR = 'T:/pycharm_repo/Working_dir/Deepdub'
DRAMA = '/tale_of_nine_tailed'

OUTPUT_DIR = './output_dir' + DRAMA
SAMPLE_DIR = BASE_DIR + DRAMA
SUBTITLE_DIR = SAMPLE_DIR + '/subtitles/'
EP = 1

In [15]:
def to_sec(min_sec):
  """
  Convert a string formatted as `min_sec` or 'h_min_sec' to int of total seconds.
  """
  return int(
    min_sec.split("_")[-2])*60 + int(
    min_sec.split("_")[-1])


print(to_sec("0_1_30")) # 1 min 30 sec
print(to_sec("1_30"))   # 1 min 30 sec
# print(to_sec("1"))    # won't make it

90
90


In [16]:
float(".433")

0.433

In [17]:
def to_sec_ms(h_min_sec_ms):
  """
  Convert a string formatted as 'h_min_sec.ms' to float of total seconds.microseconds.
  """
  try:
    ms = float("." + h_min_sec_ms.split("_")[2].split(".")[1])
  except IndexError:
    ms = float(0)
  return float(
    h_min_sec_ms.split("_")[0])*3600 + int(
    h_min_sec_ms.split("_")[1])*60 + int(
    h_min_sec_ms.split("_")[2].split(".")[0]) + ms


# print(to_sec_ms("1_30")  )    # won't make it
print(to_sec_ms("0_1_30.900"))  # 1 min 30 sec 900 ms
print(to_sec_ms("0_1_30"))      # 1 min 30 sec
print(to_sec_ms("1_1_30"))      # 1 hr 1 min 30 sec

90.9
90.0
3690.0


In [18]:
def timestamp_to_seconds(timestamp):
  return timestamp.hour + timestamp.second + timestamp.microsecond/1000000

In [19]:
def play_audio(path):
  return display.display(display.Audio(path))

In [20]:
def play_video(path):
  display.display(display.Video(path))

In [21]:
Path(f"./output_dir{DRAMA}/audio_segments").mkdir(parents=True, exist_ok=True)

### Create a sample clip using `ffmpeg_tools` in moviepy package

In [22]:
slice_from = "10_33"
slice_to = "11_00"

# min_sec
starttime = to_sec("10_33")
endtime = to_sec("11_00")

In [23]:
# Replace the filename below.
video_file = SAMPLE_DIR + f"/ep{EP}.mp4"
clipped_file = OUTPUT_DIR + f"/clip{EP}.mp4"

ffmpeg_extract_subclip(video_file, starttime, endtime, targetname=clipped_file)

Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful


In [24]:
play_video(clipped_file)

In [32]:
# Creating time accurate video clip
# https://stackoverflow.com/a/21422776/8277795
from moviepy.config import get_setting
from moviepy.tools import subprocess_call

# Replace the filename below.
video_file = SAMPLE_DIR + f"/ep{EP}.mp4"
clipped_file = OUTPUT_DIR + f"/clip{EP}.mp4"


def extract_subclip(filename, starttime, endtime, targetname):
  name, ext = os.path.splitext(filename)
  cmd = [get_setting("FFMPEG_BINARY"), "-y",
         "-ss", "%0.2f" % starttime,
         "-i", filename,
         "-t", "%0.2f" % (endtime-starttime),
         "-map", "0", "-vcodec", "copy", "-acodec", "copy", targetname]
  subprocess_call(cmd)

In [25]:
# Extract audio from video
# This is working pretty fine
output_audio = OUTPUT_DIR + f'/clip{EP}.mp3'
ffmpeg_extract_audio(clipped_file, output_audio)

play_audio(output_audio)

Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful


# Clip Audio based on Sentences and extract non sentence speaking audio too.

In [26]:
deep_s = DeepdubSentence(SUBTITLE_DIR + f'ep{EP}_eng.srt',
                          slice_from=slice_from, slice_to=slice_to)
sentence_df = deep_s.get_sentences()
sentence_df

,start,end,sentence
hash,,,
11264188807342635219,1900-01-01 00:00:14.373,1900-01-01 00:00:16.342,Why is it raining? I didn't see this on the we...
6644551431160355997,1900-01-01 00:00:16.342,1900-01-01 00:00:18.543,"Gosh, I know. I got my hair done today."
6924691847184957568,1900-01-01 00:00:18.743,1900-01-01 00:00:20.712,"- It's all wet. - My goodness, you're right."
17775764441792456539,1900-01-01 00:00:21.383,1900-01-01 00:00:22.482,Your hair is all wet.
5738569255861897594,1900-01-01 00:00:22.812,1900-01-01 00:00:24.653,That's because a fox is getting married today.


In [29]:
print(sentence_df.iloc[0].start.hour)
print(sentence_df.iloc[0].start.second)
print(sentence_df.iloc[0].start.microsecond/1000000)
timestamp_to_seconds(sentence_df.iloc[0].start)

0
14
0.373


14.373

In [37]:
AUDIO_OUTPUT_DIR = OUTPUT_DIR + '/audio_segments'


def clip_audio(inputfile, clip_from, clip_to, output,  bitrate=3000, fps=44100):
  """Clip audio from another audio file and save it in `output`"""
  cmd = [get_setting("FFMPEG_BINARY"), "-y",
         "-ss", "%0.3f" % clip_from,
         "-i", inputfile,
         "-t", "%0.3f" % (clip_to-clip_from),
         "-ab", "%dk" % bitrate,
         "-ar", "%d" % fps, output]
  subprocess_call(cmd)


sentence_df_row = 0
sub_audio_clip = AUDIO_OUTPUT_DIR + f'/{sentence_df.iloc[sentence_df_row].name}.mp3'

clip_audio(output_audio,
           timestamp_to_seconds(sentence_df.iloc[sentence_df_row].start),
           timestamp_to_seconds(sentence_df.iloc[sentence_df_row].end),
           sub_audio_clip)

play_audio(sub_audio_clip)

Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful


#### `audio_df` will contains information we need to use to clip audio.
Both where the sentence were spoken and also where they weren't.


- Where the sentences are not spoken, there is background sound effect in play. That is important.

- Where sentences are spoken, we need to extract that background sound effect to avoid it as "Noise".

At last we can then just combine these audio clips using the same `audio_df` to generate the final audio file for video

In [39]:
vals = sentence_df.reset_index()[['start', 'end']].stack().unique()
vals2 = np.concatenate([np.array([0], dtype=vals.dtype), vals])
audio_df = pd.DataFrame(zip(vals2, vals), columns=['start', 'end'])
audio_df

,start,end
0,1970-01-01 00:00:00.000,1900-01-01 00:00:14.373
1,1900-01-01 00:00:14.373,1900-01-01 00:00:16.342
2,1900-01-01 00:00:16.342,1900-01-01 00:00:18.543
3,1900-01-01 00:00:18.543,1900-01-01 00:00:18.743
4,1900-01-01 00:00:18.743,1900-01-01 00:00:20.712
5,1900-01-01 00:00:20.712,1900-01-01 00:00:21.383
6,1900-01-01 00:00:21.383,1900-01-01 00:00:22.482
7,1900-01-01 00:00:22.482,1900-01-01 00:00:22.812
8,1900-01-01 00:00:22.812,1900-01-01 00:00:24.653


In [40]:
# The above DataFrame doesn't have information about
# last clip, after all the sentences has ended

from pyffmpeg import FFprobe

audio_df = pd.concat([
  audio_df, pd.DataFrame({
    "start": [audio_df.iloc[-1].end],
    "end": [pd.to_datetime(FFprobe(output_audio).duration, format="%H:%M:%S.%f")]
  })], ignore_index=True, axis=0)
audio_df["hash"] = pd.util.hash_pandas_object(audio_df[["start", "end"]], index=False)
audio_df

,start,end,hash
0,1970-01-01 00:00:00.000,1900-01-01 00:00:14.373,10959502890968151517
1,1900-01-01 00:00:14.373,1900-01-01 00:00:16.342,11264188807342635219
2,1900-01-01 00:00:16.342,1900-01-01 00:00:18.543,6644551431160355997
3,1900-01-01 00:00:18.543,1900-01-01 00:00:18.743,7555501203862005161
4,1900-01-01 00:00:18.743,1900-01-01 00:00:20.712,6924691847184957568
5,1900-01-01 00:00:20.712,1900-01-01 00:00:21.383,4319745680707423605
6,1900-01-01 00:00:21.383,1900-01-01 00:00:22.482,17775764441792456539
7,1900-01-01 00:00:22.482,1900-01-01 00:00:22.812,9127085250858060588
8,1900-01-01 00:00:22.812,1900-01-01 00:00:24.653,5738569255861897594
9,1900-01-01 00:00:24.653,1900-01-01 00:00:27.060,5485126294294335857


In [41]:
def create_audio_clips(audio_df, output_dir):
  for index, row in audio_df.iterrows():
    audio_clip_path = output_dir + f'/{row.hash}.mp3'
    clip_audio(output_audio,
               timestamp_to_seconds(row["start"]),
               timestamp_to_seconds(row["end"]),
               audio_clip_path)


create_audio_clips(audio_df, OUTPUT_DIR + '/audio_segments')

Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful


In [58]:
audio_df[["path"]] = audio_df[["hash"]].applymap(
  lambda h: str(h) + ".mp3" if h in sentence_df.index else str(h) + ".mp3")
audio_df

,start,end,hash,path
0,1970-01-01 00:00:00.000,1900-01-01 00:00:14.373,10959502890968151517,10959502890968151517.mp3
1,1900-01-01 00:00:14.373,1900-01-01 00:00:16.342,11264188807342635219,11264188807342635219.mp3
2,1900-01-01 00:00:16.342,1900-01-01 00:00:18.543,6644551431160355997,6644551431160355997.mp3
3,1900-01-01 00:00:18.543,1900-01-01 00:00:18.743,7555501203862005161,7555501203862005161.mp3
4,1900-01-01 00:00:18.743,1900-01-01 00:00:20.712,6924691847184957568,6924691847184957568.mp3
5,1900-01-01 00:00:20.712,1900-01-01 00:00:21.383,4319745680707423605,4319745680707423605.mp3
6,1900-01-01 00:00:21.383,1900-01-01 00:00:22.482,17775764441792456539,17775764441792456539.mp3
7,1900-01-01 00:00:22.482,1900-01-01 00:00:22.812,9127085250858060588,9127085250858060588.mp3
8,1900-01-01 00:00:22.812,1900-01-01 00:00:24.653,5738569255861897594,5738569255861897594.mp3
9,1900-01-01 00:00:24.653,1900-01-01 00:00:27.060,5485126294294335857,5485126294294335857.mp3


In [73]:
output_file = OUTPUT_DIR + '/audio_segments/audio_segments_list.txt'
with open(output_file, 'w') as fout:
    for i, row in audio_df.iterrows():
        fout.write(f'file {row["path"]}\n')

In [74]:
# Let's concatenate all our audio clips back to test it
# using `audio_clip_list.txt`
def concatenate_audio_clips():
  cmd = [get_setting("FFMPEG_BINARY"), "-y",
         "-f", "concat",
         "-safe", "0",
         "-i", OUTPUT_DIR + '/audio_segments/audio_segments_list.txt',
         "-c", "copy", 
         OUTPUT_DIR + '/audio_segments/audio_gen.mp3']
  subprocess_call(cmd)
  
concatenate_audio_clips()

Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful


In [76]:
# Then merge video clip with that generated audio
def merge_video_audio(clipped_video, output_audio):
  output_inter_video = OUTPUT_DIR + '/clip_no_sound.mp4'
  cmd = [get_setting("FFMPEG_BINARY"), "-y",
         "-i", clipped_video, 
         "-an",
         "-c", "copy",
         output_inter_video]
  subprocess_call(cmd)
  
  ffmpeg_merge_video_audio(output_inter_video, OUTPUT_DIR + '/audio_segments/audio_gen.mp3', 
                           OUTPUT_DIR + '/clip_gen.mp4')
  os.remove(output_video)

merge_video_audio(clipped_file, output_audio)

Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful


In [77]:
# Sounds good...
play_video(OUTPUT_DIR + '/clip_gen.mp4')

#### Can we fill ranges for which sentences don't exist?

In [41]:
pd.DataFrame( 
{'start': 
  {0: pd.Timestamp('1900-01-01 00:00:14.373000'), 1: pd.Timestamp('1900-01-01 00:00:16.342000'),2: pd.Timestamp('1900-01-01 00:00:18.743000'), 3: pd.Timestamp('1900-01-01 00:00:21.383000'), 4: pd.Timestamp('1900-01-01 00:00:22.812000')}, 
'end': 
  {0: pd.Timestamp('1900-01-01 00:00:16.342000'), 1: pd.Timestamp('1900-01-01 00:00:18.543000'), 2: pd.Timestamp('1900-01-01 00:00:20.712000'), 3: pd.Timestamp('1900-01-01 00:00:22.482000'), 4: pd.Timestamp('1900-01-01 00:00:24.653000')}})

,start,end
0,1900-01-01 00:00:14.373,1900-01-01 00:00:16.342
1,1900-01-01 00:00:16.342,1900-01-01 00:00:18.543
2,1900-01-01 00:00:18.743,1900-01-01 00:00:20.712
3,1900-01-01 00:00:21.383,1900-01-01 00:00:22.482
4,1900-01-01 00:00:22.812,1900-01-01 00:00:24.653


In [46]:
# We need to create this dataframe
# Got it https://stackoverflow.com/a/71481506/8277795
pd.DataFrame( 
{'start': 
  {0: pd.Timestamp('1900-01-01 00:00:00.000000'), 1: pd.Timestamp('1900-01-01 00:00:14.373000'), 2: pd.Timestamp('1900-01-01 00:00:16.342000'), 3: pd.Timestamp('1900-01-01 00:00:18.543000'), 4: pd.Timestamp('1900-01-01 00:00:20.712000'), 5: pd.Timestamp('1900-01-01 00:00:21.383000'), 6: pd.Timestamp('1900-01-01 00:00:22.482000'), 7: pd.Timestamp('1900-01-01 00:00:22.812000')}, 
'end': 
  {0: pd.Timestamp('1900-01-01 00:00:14.373000'), 1: pd.Timestamp('1900-01-01 00:00:16.342000'), 2: pd.Timestamp('1900-01-01 00:00:18.543000'), 3: pd.Timestamp('1900-01-01 00:00:20.712000'), 4: pd.Timestamp('1900-01-01 00:00:21.383000'), 5: pd.Timestamp('1900-01-01 00:00:22.482000'), 6: pd.Timestamp('1900-01-01 00:00:22.812000'), 7: pd.Timestamp('1900-01-01 00:00:24.653000')}})

,start,end
0,1900-01-01 00:00:00.000,1900-01-01 00:00:14.373
1,1900-01-01 00:00:14.373,1900-01-01 00:00:16.342
2,1900-01-01 00:00:16.342,1900-01-01 00:00:18.543
3,1900-01-01 00:00:18.543,1900-01-01 00:00:20.712
4,1900-01-01 00:00:20.712,1900-01-01 00:00:21.383
5,1900-01-01 00:00:21.383,1900-01-01 00:00:22.482
6,1900-01-01 00:00:22.482,1900-01-01 00:00:22.812
7,1900-01-01 00:00:22.812,1900-01-01 00:00:24.653


In [52]:
vals = sentence_df.reset_index()[['start', 'end']].stack().unique()
vals2 = np.concatenate([np.array([0], dtype=vals.dtype), vals])
audio_df = pd.DataFrame(zip(vals2, vals), columns=['start', 'end'])
audio_df

,start,end
0,1970-01-01 00:00:00.000,1900-01-01 00:00:14.373
1,1900-01-01 00:00:14.373,1900-01-01 00:00:16.342
2,1900-01-01 00:00:16.342,1900-01-01 00:00:18.543
3,1900-01-01 00:00:18.543,1900-01-01 00:00:18.743
4,1900-01-01 00:00:18.743,1900-01-01 00:00:20.712
5,1900-01-01 00:00:20.712,1900-01-01 00:00:21.383
6,1900-01-01 00:00:21.383,1900-01-01 00:00:22.482
7,1900-01-01 00:00:22.482,1900-01-01 00:00:22.812
8,1900-01-01 00:00:22.812,1900-01-01 00:00:24.653
